# Ranting about $K_d$

In [10]:
# imports
from importlib import reload
from importlib.resources import files
import os
import numpy as np

from scipy.optimize import curve_fit

import pandas
import seaborn as sns
from matplotlib import pyplot as plt

from scipy.interpolate import interp1d

from ocpy.hydrolight import loisel23
from ocpy.utils import plotting as oc_plotting
from ocpy.satellites import pace as sat_pace
from ocpy.water import absorption as water_abs

from bing.models import anw as boring_anw
from bing.models import bbnw as boring_bbnw
from bing.models import utils as model_utils
from bing.models import functions
from bing import inference as big_inf
from bing import rt as big_rt
from bing import chisq_fit


In [2]:
%matplotlib ipympl

# Load up Loisel+2023 Hydrolight dataset

## Using the full run (inelastic scattering) at Zenith

In [3]:
ds = loisel23.load_ds(4,0)

In [4]:
l23_wave = ds.Lambda.data
l23_Rrs = ds.Rrs.data
all_a = ds.a.data
all_b = ds.b.data
all_bnw = ds.bnw.data
all_bbnw = ds.bbnw.data
all_bb = ds.bb.data
all_adg = ds.ag.data + ds.ad.data
all_ad = ds.ad.data
all_ag = ds.ag.data
all_aph = ds.aph.data
all_anw = ds.anw.data
#
all_bw = all_b - all_bnw
#
all_bbd = ds.bbd.data
all_bbph = ds.bbph.data

In [19]:
l23_aw = all_a[0] - all_anw[0]
l23_bbw = all_bb[0] - all_bbnw[0]

In [6]:
all_anw.shape

(3320, 81)

In [8]:
l23_wave.min(), l23_wave.max()

(350.0, 750.0)

## Setup

In [18]:
i350 = np.argmin(np.abs(l23_wave-350.))
i440 = np.argmin(np.abs(l23_wave-440.))
i750 = np.argmin(np.abs(l23_wave-750.))  # Should have been 780

## More water

In [14]:
more_wave = np.arange(320., 781., 1.)
aw = water_abs.a_water(more_wave)
#
i780 = np.argmin(np.abs(more_wave-780.))  

# Coefficients for Hooker+2013

## Express their expression as:

## $a_{\rm CDOM}(440) = H_0 \left [ \frac{K_d(320)}{K_d(780)} \right ] - H_1  \;\; [\rm m^{-1}]$

## with $H_0 = 0.293$ and $H_1 = 0.015$

## Starting with $H_0$

In [17]:
bb780 = 0.
H_0 = aw[i780] + 4.18 * (1 - 0.52 * np.exp(-10.8*aw[i780])) * bb780
H_0

2.6905

### Hmm.. that's far off

### What did I mess up??

## Now $H_1$

### We will assume that water dominates back-scattering

### And that $a_{\rm CDOM}(320)$ is larger enough to ignore the term in the brackets

### I don't have the Zhang MatLab code ported to Python yet (bug)

### So we'll use $b_{\rm b,w}(350)$ from L23

In [20]:
H1 = 4.18 * l23_bbw[i350]
H1

0.02473431401886046

### That's about $2 \times$ larger than Hooker+2013, but not a total "disaster"